In [2]:
import pandas
df = pandas.read_csv('lichess_db_puzzle.csv.zst')

In [23]:
# get all possible
all_themes = set([])
for theme in df['Themes']:
    all_themes.update(theme.split(" "))

In [43]:
# assign an index to each
len(all_themes)
ref_themes = {}
for idx, theme in enumerate(all_themes):
    ref_themes[theme] = idx

# change each row of themes from list to array of bools indicating which themse
pp_themes = []
for theme in df['Themes']:
    list_o_themes = theme.split(" ")
    checklist = [0 for _ in range(60)]
    for ind_theme in list_o_themes:
        checklist[ref_themes[ind_theme]] = 1
    pp_themes.append(checklist)

33

In [48]:
# want to use FEN, Moves, Rating, and RatingDeviation to predict Themes
import numpy as np
from sklearn.model_selection import train_test_split



cols = ["Rating", "RatingDeviation", "Themes"]
X_train, X_test, y_train, y_test = train_test_split(df[['Rating', 'RatingDeviation']], pp_themes, test_size=0.33, random_state=0)


from sklearn import preprocessing

X_train_norm = preprocessing.normalize(X_train)
X_test_norm = preprocessing.normalize(X_test)

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters = 3, random_state = 0, n_init='auto')
kmeans.fit(X_train_norm)


KMeans(n_clusters=3, n_init='auto', random_state=0)

In [76]:
label = kmeans.fit_predict(X_test_norm)
print(label)

[2 0 0 ... 1 2 2]


In [77]:
def get_key(val):
    for key, value in ref_themes.items():
        if val == value:
            return key

    return "key doesn't exist"

def accuracy(labels, y_test):
    if len(labels) != len(y_test):
        return "Unequal lengths"
    corrects = 0
    for i in range(len(label)):
        correct_labels = set([])
        cur = y_test[i]
        for j in range(len(cur)):
            if cur[j] == 1:
                correct_labels.add(j)
        if labels[i] in correct_labels:
            corrects+=1
    return corrects/len(label) * 100

accuracy(label, y_test)

8.023273161271073

### Ok thats really bad -> results: ~8% accuracy
#### as expected, predicting the themes solely based on the rating and rating distribution is idiotic